In [17]:
from cdlib import algorithms
import networkx.algorithms.community as nx_comm
from cdlib import viz
import networkx as nx
from cdlib import NodeClustering
import random
import numpy as np
import pandas as pd
from itertools import product
import time
import sys

In [18]:


def reformat2(G,coms): 
    hasil=[]
    for key in coms: #a_dict[key])
        if len(coms[key])>0:
            hasil.append(list(coms[key]))
    return(hasil)



In [19]:
def cetak(s, communities):
    comm= []
    for x in communities:
        if list(x)!=[]:
            comm.append(list(x))
    print (s,' : ',comm)


In [20]:
def intersection(lst1, lst2):
    #https://stackoverflow.com/questions/54610818/difference-between-two-list-of-unordered-tuples
    if lst1 and lst2:
        lst3= set([tuple(sorted(elem))for elem in lst1]) & set([tuple(sorted(elem))for elem in lst2])
    else:
        lst3=[]
    return lst3


In [21]:
def pecah(G,communities):
    # ini draft awal pecah 
    cand=[]
    for idx in range(len(communities)):
        if (len(communities[idx])>1): # Apakah anggota comunitas berjumlah dua atau lebih
            cand.append(idx)
    if len(cand)>=1:  # bila ada yang panjangnya dua atau lebih
        terpilih = random.choices(cand)  # perlu diperbaiki, kriteria yang dipecah 
        terpilihscalar=terpilih[0]
        elemen=list(communities[terpilihscalar])
        for el in elemen:
            communities[el]=frozenset({el})

In [22]:
# cari community yang dipecah secara random
def pecahrandom(G,communities):
    commr=reformat2(G,communities)
    cand=[]
    for idx in range(len(communities)):
        if (len(communities[idx])>1): 
            # Apakah anggota comunitas berjumlah dua atau lebih
            cand.append(idx)
    if len(cand)>=1:  # bila ada yang panjangnya dua atau lebih
        terpilih = random.choices(cand)  # random         
        terpilihscalar=terpilih[0]
        elemen=list(communities[terpilihscalar])
        for el in elemen:
            communities[el]=frozenset({el})
    commr=reformat2(G,communities)

In [23]:
def lepasnodelemah(G,communities):
    kom_ke=0
    for idx in range(len(communities)):
        if len(communities[idx])>0:
            kom_ke+=1
            list_node_komunitas = [x for x in  communities[idx]]
            for node_komunitas in list_node_komunitas:
                list_edge_node_komunitas=list(G.edges(node_komunitas))
                in_comm = 0
                out_comm= 0
                for e in list_edge_node_komunitas:
                    if e[1] in list_node_komunitas:
                        in_comm += 1
                    else:
                        out_comm += 1
               
                #print(node_komunitas, G.degree[node_komunitas], in_comm, out_comm,nodelemah,lepas)
                if in_comm>=2 and out_comm>in_comm:
                    #print("kandidat node yang dilepas :", node_komunitas,"pada komunitas ke :", idx)
                    elemen=list(communities[idx])
                    communities[idx]=frozenset()
                    #print("elemen komunitas yang akan dilepas", elemen)
                    for el in elemen:
                        communities[el]=frozenset()
                    sisa=[item for item in elemen if item != node_komunitas]
                    communities[node_komunitas]=frozenset({node_komunitas})
                    subgraphsisa =  G.subgraph(sisa)
                    listconns = [list(c) for c in nx.connected_components(subgraphsisa)]
                    for  listconn in listconns:
                        communities[listconn[0]]=frozenset(listconn)
                    break;     
          

In [24]:
def greedy_modularity_communities_r3c(G, mode,modelepas,multiply):
    global dfcom 
    
    def filter_node(n1):
        return n1 in gabung
    #inisialisasi
    communities = {n: frozenset([n]) for n in G}
    numiter =G.number_of_nodes()*multiply
    #numiter =50
    
    numedges=len(G.edges())
    
    maksdq=1
    iter=0
    maxmodularity=0
    maxcommunities=communities
    keluar=False
    recount=0

    cek=False
    # iterasi dimulai disini 
    nnodes= G.number_of_nodes()
    while True:
        awal= time.time()
        iter=iter+1
        #print(iter)
        urutan = np.random.choice([1,2,3],1, p=[0.9,0.05,0.05])
        pilihan=urutan[0]
        #if iter==1:
        if iter<=nnodes:    
            pilihan=1
        if pilihan==1 or recount==1: 
            if iter==1 or recount==1:
                df = pd.DataFrame(columns = ['i', 'j', 'u','v', 'eij', 'ai','aj','dq'])
                elc=list()
                lgec=list()
                for k in range(len(communities)):
                    elc.append(G.edges(list(communities[k])))
                    lgec.append(len(G.edges(communities[k])))
                for i in range(len(communities)):
                    for j in range(len(communities)):
                        if j<=i:
                            continue
                        elci = elc[i]
                        elcj = elc[j]
                        join=intersection(elci,elcj)

                        if len(join)==0:
                            continue
                        eij=len(join)/numedges    
                        ai = lgec[i]/numedges
                        aj = lgec[j]/numedges
                        dq= 2*(eij-ai*aj)
                        df = df.append({'i' : i, 'j' : j, 'u' : communities[i], 'v' :communities[j], 'eij':eij, 'ai':ai, 'aj':aj,'dq':dq},ignore_index = True)            
                recount=0
            rowmaxdq=df['dq'].idxmax()
            #print("baris maksimal adalah", rowmaxdq, "yaitu community ke", df["i"][rowmaxdq]," dan ",df["j"][rowmaxdq])
            # yang digabung adalah df["i"][rowmaxdq] dan df["j"][rowmaxdq]
            maksdq=df["dq"][rowmaxdq]
            if maksdq>0:
                frozenUnion = communities[df["i"][rowmaxdq]].union(communities[df["j"][rowmaxdq]])
                #print(frozenUnion)
                if len(frozenUnion)==0:
                    df.to_excel("cek error.xlsx")
                    print('iterasi', iter)
                    print('dataframe')
                    print(df)
                    print('rowmaxdq,i,j:',rowmaxdq)
                    print(df["i"][rowmaxdq])
                    print(df["j"][rowmaxdq])
                    print('frozenunion:',communities[df["i"][rowmaxdq]],communities[df["j"][rowmaxdq]],frozenUnion, len(frozenUnion))
                    print(communities)
                    
                indminfrozen=min(frozenUnion)
                if indminfrozen==df["i"][rowmaxdq]:
                    communities[df["i"][rowmaxdq]]=frozenUnion
                    communities[df["j"][rowmaxdq]]=frozenset()
                else:
                    communities[df["i"][rowmaxdq]]=frozenset()
                    communities[df["j"][rowmaxdq]]=frozenUnion
            # buat communities baru dengan menggabungkan baris tersebut
            rowmaxdq=df['dq'].idxmax()
            udel=df["u"][rowmaxdq]
            vdel=df["v"][rowmaxdq]
            listdrop=[rowmaxdq]
            listadd=[]
            for i in range(len(df)):
                if i!= rowmaxdq:
                    
                    if (df.loc[i, "u"]== udel) or (df.loc[i, "u"]==vdel):  
                        #print(i," yang dihapus ",df.loc[i, "u"], " yang ditambahkan",df.loc[i, "v"])
                        if df.loc[i, "v"] not in listadd:
                            listadd.append(df.loc[i, "v"])
                        if i not in listdrop:
                            listdrop.append(i)
                    if (df.loc[i, "v"]== udel) or (df.loc[i, "v"]== vdel):
                        #print(i," yang dihapus ",df.loc[i, "v"],"yang ditambahkan",df.loc[i, "u"])
                        if df.loc[i, "u"] not in listadd:
                            listadd.append(df.loc[i, "u"])
                        if i not in listdrop:
                            listdrop.append(i)
            #print("listdrop :", listdrop)
            df.drop(listdrop,0,inplace=True)
            #print("listadd :", listadd)
            i= min(frozenUnion)
            elci = G.edges(list(frozenUnion))
            for ellistadd in listadd:
                j=min(ellistadd)
                elcj = G.edges(list(ellistadd))
                join=intersection(elci,elcj)
                eij=len(join)/numedges 
                ai = len(G.edges(list(frozenUnion)))/numedges
                aj = len(G.edges(list(ellistadd)))/numedges                         
                dq= 2*(eij-ai*aj)  
                df=df.append({'i' : i, 'j' : j, 'u' : frozenUnion, 'v' :ellistadd, 'eij':eij, 'ai':ai, 'aj':aj,'dq':dq},ignore_index = True)            
            comsr=reformat2(G,communities)
            modularity = nx_comm.modularity(G,comsr)
            
            if modularity>maxmodularity:
                maxmodularity=modularity
                maxcommunities= communities.copy()
            if df.empty:
                keluar=True
            else:
                rowmaxdq=df['dq'].idxmax()
                maksdq=df["dq"][rowmaxdq]
                if maksdq<=0:
                    keluar=True
            if keluar and iter<numiter:
                if iter <numiter:
                    pilihan=3
                else:
                    break
            if iter>=numiter: #or keluar:
                break
        if pilihan==2:
            if modelepas==0:
                lepasnodelemah(G, communities)
            elif modelepas==1:
                lepasnodenontpn(G,communities)
            elif modelepas==2:
                lepasnodelowembeddedness(G,communities)
        if pilihan==3:
            if keluar:
                keluar=False
            if mode==0:
                pecahrandom(G, communities)
            elif mode==1:
                pecahweak(G,communities)
            elif mode==2:
                pecahconductance(G,communities)
            elif mode==3:
                pecahtpr(G,communities)
            elif mode==4:
                pecahinternaledge(G,communities)
            recount=1
            cek=True
        akhir= time.time()
        selisih=akhir-awal           
        dftemp = {'file':file, 
                  'simpul':len(G.nodes()),
                  'iterasi': iter,
                  'mode':smode,
                  'modelepas':smodelepas,
                  'modularity':modularity,
                  'maxmodularity':maxmodularity,
                  'n_komunitas':len(comsr),
                  'time':selisih}
        dfcom = dfcom.append(dftemp,ignore_index=True)          
    return maxcommunities, iter


In [26]:
# beberapa ujicoba
#G=sampleuji.sample8()

files=["protein.gml"]
#files=["LFR 250 3 1_5 0_1.gml"]
for file in files:
    
    G = nx.read_gml(file, label = 'id') 
    #G=nx.Graph()
    #G.add_edges_from([(0,1),(1,2),(2,0),(2,3),(3,4),(4,5),(5,3),(6,5),(6,3)])
    G=nx.convert_node_labels_to_integers(G,first_label=0)
    pos = nx.spring_layout(G)
    mode=0
    modelepas = 2
    smode='pecah_random'
    smodelepas='low embeddedness'
    dfcom = pd.DataFrame(columns = [ 'file','simpul','iterasi','mode','modelepas','modularity', 'n_komunitas','time'])
    G=nx.convert_node_labels_to_integers(G,first_label=0)
    #for step in range (90,100):
    awal = time.time()
    coms, iter=greedy_modularity_communities_r3c(G,0,0,4)
    akhir= time.time()
    comsr=reformat2(G,coms)
    #print(coms)
    modcomsr = nx_comm.modularity(G,comsr)
    #print("modularity", modcomsr)
    selisih= akhir-awal
    print("greedy_disassembly",file, len(G.nodes()), modcomsr,selisih, iter)
# 0.40203813280736356
#dfcom.to_excel("LFR.xlsx")

greedy_disassembly protein.gml 703 0.4685882178554962 2043.8852953910828 2812


In [27]:
modcomsr

0.4685882178554962

In [19]:
dfcom


,file,simpul,iterasi,mode,modelepas,modularity,n_komunitas,time,maxmodularity
0,Gene NCBI.gml,1258,1,pecah_random,low embeddedness,-0.001905,1257,25.511833,0.000000
1,Gene NCBI.gml,1258,2,pecah_random,low embeddedness,-0.001754,1256,0.206415,0.000000
2,Gene NCBI.gml,1258,3,pecah_random,low embeddedness,-0.001609,1255,0.209408,0.000000
3,Gene NCBI.gml,1258,4,pecah_random,low embeddedness,-0.001471,1254,0.207473,0.000000
4,Gene NCBI.gml,1258,5,pecah_random,low embeddedness,-0.001333,1253,0.206418,0.000000
...,...,...,...,...,...,...,...,...,...
5026,Gene NCBI.gml,1258,5027,pecah_random,low embeddedness,0.331887,650,0.140594,0.571911
5027,Gene NCBI.gml,1258,5028,pecah_random,low embeddedness,0.332320,649,0.085801,0.571911
5028,Gene NCBI.gml,1258,5029,pecah_random,low embeddedness,0.332752,648,0.087737,0.571911
5029,Gene NCBI.gml,1258,5030,pecah_random,low embeddedness,0.333040,647,0.082804,0.571911


In [28]:
#coms
#coms_formatted = [list(coms) for coms in comsf.values() if coms]
coms_formatted = [list(com) for com in coms.values() if com]


In [29]:
arrgene = [G.nodes[node]["label"] for node in sorted(G.nodes)]
arrgene

['ABI1',
 'MAPK1',
 'MAPK3',
 'PIK3CA',
 'EGFR',
 'PIK3CB',
 'RAC1',
 'PIK3CG',
 'ABL1',
 'PIK3R1',
 'WAS',
 'SOS1',
 'CCND1',
 'CDKN1B',
 'PLCG1',
 'CDK4',
 'MDM2',
 'CCND2',
 'PDGFRB',
 'CCNE1',
 'CBL',
 'TP63',
 'CDK6',
 'RB1',
 'TP53',
 'ATM',
 'STAT5B',
 'RUNX1',
 'BCR',
 'TERT',
 'PDGFB',
 'HSP90AA1',
 'LCK',
 'STAT5A',
 'ATR',
 'CTNNB1',
 'CRKL',
 'FYN',
 'NUP214',
 'CBFB',
 'YES1',
 'LYN',
 'CTNND1',
 'CCND3',
 'WWTR1',
 'ABL2',
 'CDKN1A',
 'CDKN2A',
 'NTRK1',
 'SRC',
 'YAP1',
 'RAD51',
 'ACKR3',
 'CXCR4',
 'ACSL3',
 'FASN',
 'ACSL6',
 'ACVR1',
 'ACVR2A',
 'INHBA',
 'SMAD2',
 'SMAD3',
 'BMPR1A',
 'BMP5',
 'INHA',
 'SMAD4',
 'AFF1',
 'MLLT1',
 'ELL',
 'MLLT10',
 'MLLT3',
 'DOT1L',
 'AFF4',
 'HEXIM1',
 'AGO2',
 'NOTCH4',
 'NUP98',
 'NUP93',
 'CLP1',
 'NOTCH3',
 'NOTCH2',
 'RANBP2',
 'CREB1',
 'TPR',
 'NOTCH1',
 'SOX2',
 'FOXO3',
 'PTEN',
 'MYC',
 'NPM1',
 'DGCR8',
 'PABPC1',
 'DROSHA',
 'EIF4E',
 'DDX6',
 'DICER1',
 'AKAP9',
 'RAF1',
 'ARAF',
 'PKN1',
 'JAK2',
 'PDE4DIP',
 'AKT1'

In [30]:
coms_formatted

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  520,
  9,
  10,
  11,
  14,
  527,
  528,
  529,
  18,
  20,
  26,
  28,
  30,
  32,
  33,
  545,
  36,
  37,
  550,
  549,
  40,
  41,
  42,
  548,
  559,
  48,
  561,
  49,
  560,
  563,
  53,
  52,
  573,
  579,
  580,
  581,
  587,
  588,
  589,
  590,
  591,
  592,
  593,
  594,
  87,
  601,
  607,
  608,
  97,
  98,
  612,
  100,
  613,
  105,
  106,
  107,
  626,
  627,
  116,
  118,
  631,
  120,
  633,
  122,
  635,
  636,
  125,
  632,
  630,
  128,
  129,
  641,
  133,
  646,
  136,
  137,
  146,
  150,
  664,
  665,
  157,
  669,
  670,
  160,
  161,
  678,
  679,
  168,
  681,
  171,
  172,
  173,
  174,
  683,
  176,
  177,
  175,
  181,
  701,
  206,
  207,
  208,
  209,
  210,
  211,
  213,
  214,
  215,
  217,
  218,
  219,
  220,
  221,
  222,
  223,
  224,
  225,
  242,
  243,
  305,
  314,
  316,
  317,
  318,
  319,
  320,
  321,
  322,
  323,
  324,
  325,
  326,
  327,
  331,
  346,
  347,
  348,
  351,
  353,
  354,
  35

In [31]:
id_to_label = {i: label for i, label in enumerate(arrgene)}

# Lakukan mapping untuk mendapatkan coms_gene
coms_gene = [[id_to_label[node_id] for node_id in community] for community in coms_formatted]
coms_gene

[['ABI1',
  'MAPK1',
  'MAPK3',
  'PIK3CA',
  'EGFR',
  'PIK3CB',
  'RAC1',
  'PIK3CG',
  'MUC1',
  'PIK3R1',
  'WAS',
  'SOS1',
  'PLCG1',
  'GNA11',
  'GNAS',
  'SDC4',
  'PDGFRB',
  'CBL',
  'STAT5B',
  'BCR',
  'PDGFB',
  'LCK',
  'STAT5A',
  'DUSP4',
  'CRKL',
  'FYN',
  'NRG1',
  'ERRFI1',
  'YES1',
  'LYN',
  'CTNND1',
  'MUC4',
  'EPHA5',
  'NTRK1',
  'EPHB1',
  'SRC',
  'EPHA7',
  'GRIN2A',
  'CXCR4',
  'ACKR3',
  'ETV6',
  'FES',
  'MSN',
  'MUC16',
  'TEK',
  'IRS2',
  'FGF19',
  'FGFR4',
  'FGF3',
  'FGF4',
  'ZMYM2',
  'FGFR1OP',
  'PTEN',
  'IL6ST',
  'SH3GL1',
  'SH2D1A',
  'RAF1',
  'ARAF',
  'TSHR',
  'JAK2',
  'PIK3C2G',
  'PIK3R2',
  'IL2',
  'RAC2',
  'INSRR',
  'IRS4',
  'CBLB',
  'STAT3',
  'INPP4A',
  'ERBB3',
  'INPP4B',
  'PAK1',
  'TNC',
  'TEC',
  'INPPL1',
  'PIK3C3',
  'IL21R',
  'INSR',
  'IRS1',
  'PTPRB',
  'PPP2R1A',
  'KTN1',
  'PTPN11',
  'JAK1',
  'RALGDS',
  'PIK3CD',
  'MYH11',
  'MYH9',
  'CDC42',
  'TFG',
  'PTPRS',
  'JAK3',
  'RHOA',
  'PPP4R2'